In [210]:
import pandas as pd
pd.set_option('display.max_columns', None)
#import matplotlib.pyplot as plt
from meteostat import Stations, Daily, Hourly
import datetime as dt
import numpy as np

In [152]:
import requests
#import urllib.parse

In [217]:
def getMeteostat(loc='03647',date_range=None,offset=0,sample_rate='daily',interpol=1,min_coverage=0.9):
    
    # Address entered - trying to fetch lat & lon
    if isinstance(loc,dict):
        
        latlong,_=address2coords(view=0,**loc)
        
        if len(latlong)>0:
            print('Successfully derived lat & lon from address data!')
            loc=list(latlong.iloc[0,:])
        else:
            print('!!! Sorry, address query did not return any lat/lon data - try different address')
            loc=None
    
    # If loc entered is valid and/or geodata search was successful
    if loc is not None:
        
        # Sample rate
        if sample_rate!='daily':
            sample_rate='hourly'
            f='H'
        else:
            f='D'
        
        # If no date range was entered, use last 365 days, with selected offset in days
        if date_range is None:
            print('Date range not provided - taking last year, offset=',offset,'days')
            end=dt.datetime.today()-dt.timedelta(days=offset)
            delta=365+offset
            start=dt.datetime.today()-dt.timedelta(days=delta)
            #start = dt.datetime(2018, 1, 1)
            #end = dt.datetime(2018, 12, 31)
            date_range=[start,end]

        print('Using time period from ',date_range[0],' to ',date_range[1],' with ',sample_rate,' frequency')

        # If loc is a list/tuple then lat & lon was entered - fetch 20 closest stations and prioritize by data 
        # coverage & distance
        if isinstance(loc,(list,tuple)):
            print('Fetching nearby stations for latitude ',loc[0],' and longitude ',loc[1])
            st= Stations()
            st=st.nearby(loc[0], loc[1])
            #st1=st.inventory(sample_rate, date_range[0])
            #st2=st.inventory(sample_rate, date_range[1])
            #st1 = st1.fetch(10).reset_index()
            #st2 = st2.fetch(10).reset_index()
            #st = pd.concat([st1,st2],axis=0,ignore_index=True)

            #st=st.drop_duplicates().reset_index(drop=True)

            st=st.fetch(20).reset_index()

            col_start=sample_rate+'_start'
            col_end=sample_rate+'_end'

            st[col_start]=pd.to_datetime(st[col_start],dayfirst=True,errors='coerce')
            st[col_end]=pd.to_datetime(st[col_end],dayfirst=True,errors='coerce')

            null_date = pd.to_datetime('1900-01-01',dayfirst=True,errors='coerce')
            st['no_range']=0
            st.loc[st[col_start].isnull(),'no_range']=1
            st[col_start]=st[col_start].fillna(null_date)
            st[col_end]=st[col_end].fillna(null_date)

            st['valid_obs']=0
            st['coverage']=0

            #display(st)

            true_rng=pd.date_range(start=date_range[0], end=date_range[1], freq=f)

            # Analyzing valid ranges returned by Stations.nearby()
            for i in range(len(st)):

                rng=pd.date_range(start=st.loc[i,col_start], end=st.loc[i,col_end], freq=f)
                cond=(rng>=date_range[0])&(rng<=date_range[1])
                st.loc[i,'valid_obs']=-np.sum(cond)
                st.loc[i,'coverage']=-np.sum(cond)/len(true_rng)


            # Checking rows with NaNs - sometimes Stations.nearby returns no data for given station
            # but Daily() or Hourly() actually returns data - check rows with no data again using Daily()/Hourly()

            if np.sum(st['no_range']==1)>0:

                ids=list(st.loc[st['no_range']==1,'id'].astype(str))
                
                print('Double-checking data availability for the following station IDs:')
                print(ids)
                
                for idd in ids:
                    if sample_rate=='daily':
                        d = Daily(idd, date_range[0], date_range[1])
                    else:
                        d = Hourly(idd, date_range[0], date_range[1])

                    #d = d.normalize()
                    #d = d.interpolate()
                    d = d.fetch()

                    if len(d)>0:
                        d=d.reset_index()
                        st.loc[st['id']==idd,'valid_obs']=-len(d)
                        st.loc[st['id']==idd,'coverage']=-len(d)/len(true_rng)
                        st.loc[st['id']==idd,col_start]=d.loc[0,'time']
                        st.loc[st['id']==idd,col_end]=d.loc[len(d)-1,'time']
            
            # Check data completeness & order by coverage (completeness) and distance
            st['adj_coverage']=st['coverage']/min_coverage
            st.loc[st['adj_coverage']<-1,'adj_coverage']=-1
            st=st.sort_values(by=['adj_coverage','distance'],ascending=True).reset_index(drop=True)

            display(st.head())

            loc=st.loc[0,'id']
        
        # Pulling the data
        print('Puling ',sample_rate,' data for location ',loc,' and time period from ',date_range[0],' to ',date_range[1])

        if sample_rate=='daily':

            # Get daily data
            print('Pulling daily data')
            data = Daily(loc, date_range[0], date_range[1])

        else:

            print('Pulling hourly data')
            data = Hourly(loc, date_range[0], date_range[1])

        if np.sum(interpol)>0:
            data = data.normalize()
            data = data.interpolate()

        data = data.fetch()
        
    else:
        data=None
        print('!!! Sorry, location info not recognized')
    
    return(data)

In [205]:
def address2coords(street='',town='',country='',postcode='',n=1,view=1,*args):
    
    '''EXAMPLES:
    address2coords(country='United kingdom',postcode='OX18 4NH')
    
    indata={'country':'united kingdom','postcode':'OX18 4NH'}
    address2coords(**indata)
    
    address2coords(town='Vresina',country='Czech republic',n=2)
    
    address2coords(postcode='74285',country='Czech republic')
    '''
    
    response=None
    out=pd.DataFrame(columns=['Latitude','Longitude'])
    
    if len(street)>0:
        qs="&treet="+street
    else:
        qs=''
    
    if len(town)>0:
        qt="&city="+town
    else:
        qt=''
        
    if len(country)>0:
        qc="&country="+country
    else:
        qc=''
        
    if len(postcode)>0:
        qp="&postalcode="+postcode
    else:
        qp=''
    
    qry= qs+ qt + qc + qp
    
    '''lq = len(qry)-1
    
    if qry[lq]=='+':
        qry=qry[0:lq]'''
    
    if np.sum(view)>0: print('Fetching lon & lat for the following query: ',qry)
    
    if len(qry)>3:
    
        ns=str(n)
        #url = "https://nominatim.openstreetmap.org/?addressdetails=1&q=" + qry +"&format=json&limit="+ns
        url="https://nominatim.openstreetmap.org/search.php?format=json&addressdetails=1&limit="+ns+qry
        if np.sum(view)>0: print('Full query: ',url)
            
        response = requests.get(url).json()

        if len(response)>0:
            print('SUCCESS - found at least one record')
            
            if n>len(response):
                n=len(response)
            
            out['Latitude']=[0]*n
            out['Longitude']=[0]*n
            
            for i in range(n):
                out.loc[i,'Latitude']=float(response[i]["lat"])
                out.loc[i,'Longitude']=float(response[i]["lon"])

        else:
            print('SORRY - no luck, try with a different query!')
            
    else:
        
        print('!!! You entered empty query')
        
    if np.sum(view)>0: 
        print('Latitude & longitude data:')
        print(out)
        print('**************** FULL RESPONSE:')
        print(response)
    
    return(out,response)

In [208]:
address2coords(country='United kingdom',postcode='OX18 4NH')

Fetching lon & lat for the following query:  &country=United kingdom&postalcode=OX18 4NH
Full query:  https://nominatim.openstreetmap.org/search.php?format=json&addressdetails=1&limit=1&country=United kingdom&postalcode=OX18 4NH
SUCCESS - found at least one record
Latitude & longitude data:
   Latitude  Longitude
0  51.80729   -1.63962
**************** FULL RESPONSE:
[{'place_id': 287758258, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'boundingbox': ['51.64729', '51.96729', '-1.79962', '-1.47962'], 'lat': '51.80729', 'lon': '-1.63962', 'display_name': 'Burford, West Oxfordshire, Oxfordshire, South East England, England, OX18 4NH, United Kingdom', 'class': 'place', 'type': 'postcode', 'importance': 0.525, 'address': {'town': 'Burford', 'city': 'West Oxfordshire', 'county': 'Oxfordshire', 'state_district': 'South East England', 'state': 'England', 'ISO3166-2-lvl4': 'GB-ENG', 'postcode': 'OX18 4NH', 'country': 'United Kingdom', 'country_code': 'gb'

(   Latitude  Longitude
 0  51.80729   -1.63962,
 [{'place_id': 287758258,
   'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
   'boundingbox': ['51.64729', '51.96729', '-1.79962', '-1.47962'],
   'lat': '51.80729',
   'lon': '-1.63962',
   'display_name': 'Burford, West Oxfordshire, Oxfordshire, South East England, England, OX18 4NH, United Kingdom',
   'class': 'place',
   'type': 'postcode',
   'importance': 0.525,
   'address': {'town': 'Burford',
    'city': 'West Oxfordshire',
    'county': 'Oxfordshire',
    'state_district': 'South East England',
    'state': 'England',
    'ISO3166-2-lvl4': 'GB-ENG',
    'postcode': 'OX18 4NH',
    'country': 'United Kingdom',
    'country_code': 'gb'}}])

In [197]:
indata={'country':'united kingdom','postcode':'OX18 4NH'}
address2coords(**indata)

Fetching lon & lat for the following query:  &country=united kingdom&postalcode=OX18 4NH
Full query:  https://nominatim.openstreetmap.org/search.php?format=json&addressdetails=1&limit=1&country=united kingdom&postalcode=OX18 4NH
SUCCESS - found at least one record
Latitude & longitude data:
   Latitude Longitude
0  51.80729  -1.63962
**************** FULL RESPONSE:
[{'place_id': 287758258, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'boundingbox': ['51.64729', '51.96729', '-1.79962', '-1.47962'], 'lat': '51.80729', 'lon': '-1.63962', 'display_name': 'Burford, West Oxfordshire, Oxfordshire, South East England, England, OX18 4NH, United Kingdom', 'class': 'place', 'type': 'postcode', 'importance': 0.525, 'address': {'town': 'Burford', 'city': 'West Oxfordshire', 'county': 'Oxfordshire', 'state_district': 'South East England', 'state': 'England', 'ISO3166-2-lvl4': 'GB-ENG', 'postcode': 'OX18 4NH', 'country': 'United Kingdom', 'country_code': 'gb'}}

(   Latitude Longitude
 0  51.80729  -1.63962,
 [{'place_id': 287758258,
   'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
   'boundingbox': ['51.64729', '51.96729', '-1.79962', '-1.47962'],
   'lat': '51.80729',
   'lon': '-1.63962',
   'display_name': 'Burford, West Oxfordshire, Oxfordshire, South East England, England, OX18 4NH, United Kingdom',
   'class': 'place',
   'type': 'postcode',
   'importance': 0.525,
   'address': {'town': 'Burford',
    'city': 'West Oxfordshire',
    'county': 'Oxfordshire',
    'state_district': 'South East England',
    'state': 'England',
    'ISO3166-2-lvl4': 'GB-ENG',
    'postcode': 'OX18 4NH',
    'country': 'United Kingdom',
    'country_code': 'gb'}}])

In [209]:
address2coords(postcode='74285',country='Czech republic',n=2)

Fetching lon & lat for the following query:  &country=Czech republic&postalcode=74285
Full query:  https://nominatim.openstreetmap.org/search.php?format=json&addressdetails=1&limit=2&country=Czech republic&postalcode=74285
SUCCESS - found at least one record
Latitude & longitude data:
   Latitude  Longitude
0  49.82476  18.122575
**************** FULL RESPONSE:
[{'place_id': 286487385, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'boundingbox': ['49.664759611515', '49.984759611515', '17.962575265016', '18.282575265016'], 'lat': '49.82475961151486', 'lon': '18.1225752650164', 'display_name': 'Vřesina u Bílovce, Vřesina, okres Ostrava-město, Moravskoslezský kraj, Moravskoslezsko, 74285, Česko', 'class': 'place', 'type': 'postcode', 'importance': 0.33499999999999996, 'address': {'suburb': 'Vřesina u Bílovce', 'village': 'Vřesina', 'municipality': 'okres Ostrava-město', 'county': 'Moravskoslezský kraj', 'state': 'Moravskoslezsko', 'postcode': '74285'

(   Latitude  Longitude
 0  49.82476  18.122575,
 [{'place_id': 286487385,
   'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
   'boundingbox': ['49.664759611515',
    '49.984759611515',
    '17.962575265016',
    '18.282575265016'],
   'lat': '49.82475961151486',
   'lon': '18.1225752650164',
   'display_name': 'Vřesina u Bílovce, Vřesina, okres Ostrava-město, Moravskoslezský kraj, Moravskoslezsko, 74285, Česko',
   'class': 'place',
   'type': 'postcode',
   'importance': 0.33499999999999996,
   'address': {'suburb': 'Vřesina u Bílovce',
    'village': 'Vřesina',
    'municipality': 'okres Ostrava-město',
    'county': 'Moravskoslezský kraj',
    'state': 'Moravskoslezsko',
    'postcode': '74285',
    'country': 'Česko',
    'country_code': 'cz'}}])

In [149]:
# https://meteostat.net/en/place/gb/burford?t=2022-03-31/2022-04-07&s=03647
# Little Rissington: 03647 daily
d=getMeteostat(loc='03647',offset=2,interpol=0)
display(d)

Date range not provided - taking last year, offset= 2 days
Using time period from  2021-04-13 13:08:39.159547  to  2022-04-13 13:08:39.159538  with  daily  frequency
Puling  daily  data for location  03647  and time period from  2021-04-13 13:08:39.159547  to  2022-04-13 13:08:39.159538
Pulling daily data


,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2021-04-14,6.1,2.1,10.0,0.0,NaN,35.0,11.7,29.6,1032.9,NaN
2021-04-15,3.8,-1.6,8.1,0.0,NaN,50.0,13.6,27.8,1033.8,NaN
2021-04-16,4.3,-0.8,8.5,0.0,NaN,52.0,11.1,27.8,1031.7,NaN
2021-04-17,5.9,-0.2,11.5,0.0,NaN,53.0,8.7,20.4,1029.4,NaN
2021-04-18,7.5,2.5,12.8,0.0,NaN,58.0,8.4,20.4,1025.2,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-04-09,4.9,0.6,9.4,0.0,NaN,303.0,16.6,33.3,1015.2,NaN
2022-04-10,6.0,0.7,11.1,0.0,NaN,171.0,12.7,31.5,1018.1,NaN
2022-04-11,8.6,3.9,13.5,0.0,NaN,112.0,23.2,50.0,1008.9,NaN


In [150]:
#https://meteostat.net/en/place/gb/burford?t=2022-03-31/2022-04-07&s=03647
# Little Rissington: 03647 hourly
d=getMeteostat(loc='03647',offset=2,interpol=0,sample_rate='hourly')
display(d)

Date range not provided - taking last year, offset= 2 days
Using time period from  2021-04-13 13:08:41.079836  to  2022-04-13 13:08:41.079826  with  hourly  frequency
Puling  hourly  data for location  03647  and time period from  2021-04-13 13:08:41.079836  to  2022-04-13 13:08:41.079826
Pulling hourly data


,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
time,,,,,,,,,,,
2021-04-13 14:00:00,9.9,0.5,52.0,0.0,NaN,272.0,7.4,16.7,1032.1,NaN,3.0
2021-04-13 15:00:00,9.9,0.5,52.0,0.0,NaN,275.0,7.4,16.7,1031.8,NaN,3.0
2021-04-13 16:00:00,10.8,0.5,49.0,0.0,NaN,333.0,7.4,14.8,1031.3,NaN,3.0
2021-04-13 17:00:00,9.6,0.7,54.0,0.0,NaN,323.0,7.4,14.8,1031.2,NaN,3.0
2021-04-13 18:00:00,8.4,1.3,61.0,0.0,NaN,322.0,7.4,14.8,1031.3,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...,...
2022-04-13 09:00:00,10.9,10.1,95.0,0.0,NaN,221.0,18.5,31.5,1014.5,NaN,4.0
2022-04-13 10:00:00,12.1,10.0,87.0,0.1,NaN,221.0,22.2,31.5,1014.9,NaN,4.0
2022-04-13 11:00:00,12.8,10.2,84.0,0.1,NaN,227.0,18.5,31.5,1015.2,NaN,4.0


In [215]:
# Witney daily
d=getMeteostat(loc=[51.7836, -1.4854],date_range=[dt.datetime(2021, 5, 1),dt.datetime(2022, 4, 1)])
display(d)

Using time period from  2021-05-01 00:00:00  to  2022-04-01 00:00:00  with  daily  frequency
Fetching nearby stations for latitude  51.7836  and longitude  -1.4854
Double-checking data availability for the following station IDs:
['03647', 'EGBP0', 'EGUH0', 'EGTI0']


,id,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,distance,no_range,valid_obs,coverage,adj_coverage
0,03649,Brize Norton,GB,ENG,03649,EGVN,51.7500,-1.5833,88.0,Europe/London,1951-08-01,2022-04-14,1951-08-02,2022-04-12,1951-01-01,2022-01-01,7703.580458,0,-336,-1.000000,-1.0
1,EGTK0,Oxford (Kidlington) / Shipton On Cherwell,GB,ENG,<NA>,EGTK,51.8369,-1.3200,82.0,Europe/London,2020-01-14,2022-04-13,2020-01-15,2022-04-11,2020-01-01,2022-01-01,12822.802157,0,-336,-1.000000,-1.0
2,03647,Little Rissington,GB,ENG,03647,<NA>,51.8667,-1.6833,215.0,Europe/London,NaT,NaT,2021-05-01,2022-04-01,NaT,NaT,16442.749133,1,-335,-0.997024,-1.0
3,03644,Fairford Royal Air Force Base,GB,ENG,03644,EGVA,51.6833,-1.7833,87.0,Europe/London,1952-06-10,2022-04-14,1952-06-11,2022-04-12,1952-01-01,2021-01-01,23350.574279,0,-336,-1.000000,-1.0
4,03658,Benson,GB,ENG,03658,EGUB,51.6167,-1.0833,63.0,Europe/London,1973-01-01,2022-04-14,1973-01-07,2022-04-12,1982-01-01,2022-01-01,33351.431117,0,-336,-1.000000,-1.0


Puling  daily  data for location  03649  and time period from  2021-05-01 00:00:00  to  2022-04-01 00:00:00
Pulling daily data


,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2021-05-01,6.5,1.6,11.0,0.0,NaN,20.0,8.3,18.5,1017.0,NaN
2021-05-02,7.9,1.0,13.7,0.0,NaN,282.0,5.5,22.2,1020.3,NaN
2021-05-03,8.6,5.7,11.0,8.5,NaN,215.0,19.9,55.5,1011.1,NaN
2021-05-04,9.0,6.0,12.0,3.0,NaN,275.0,21.5,55.5,1000.0,NaN
2021-05-05,5.8,2.2,11.0,3.4,NaN,263.0,8.7,25.9,1009.1,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-03-28,9.6,1.0,16.0,0.0,NaN,74.0,5.9,16.7,1021.8,NaN
2022-03-29,8.8,7.0,11.0,0.3,NaN,51.0,14.3,31.5,1011.6,NaN
2022-03-30,5.8,2.0,8.0,2.2,NaN,19.0,14.6,38.9,1006.5,NaN


In [218]:
# Postcode qury daily
address={'country':'united kingdom','postcode':'OX18 4NH'}
d=getMeteostat(loc=address,date_range=[dt.datetime(2021, 5, 1),dt.datetime(2022, 4, 1)])
display(d)

SUCCESS - found at least one record
Successfully derived lat & lon from address data!
Using time period from  2021-05-01 00:00:00  to  2022-04-01 00:00:00  with  daily  frequency
Fetching nearby stations for latitude  51.80729  and longitude  -1.63962
Double-checking data availability for the following station IDs:
['03647', 'EGBP0', 'EGUH0']


,id,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,distance,no_range,valid_obs,coverage,adj_coverage
0,03647,Little Rissington,GB,ENG,03647,<NA>,51.8667,-1.6833,215.0,Europe/London,NaT,NaT,2021-05-01,2022-04-01,NaT,NaT,7255.844198,1,-335,-0.997024,-1.0
1,03649,Brize Norton,GB,ENG,03649,EGVN,51.7500,-1.5833,88.0,Europe/London,1951-08-01,2022-04-14,1951-08-02,2022-04-12,1951-01-01,2022-01-01,7456.144457,0,-336,-1.000000,-1.0
2,03644,Fairford Royal Air Force Base,GB,ENG,03644,EGVA,51.6833,-1.7833,87.0,Europe/London,1952-06-10,2022-04-14,1952-06-11,2022-04-12,1952-01-01,2021-01-01,16968.616452,0,-336,-1.000000,-1.0
3,EGTK0,Oxford (Kidlington) / Shipton On Cherwell,GB,ENG,<NA>,EGTK,51.8369,-1.3200,82.0,Europe/London,2020-01-14,2022-04-13,2020-01-15,2022-04-11,2020-01-01,2022-01-01,22212.886498,0,-336,-1.000000,-1.0
4,EGBJ0,Gloucestershire / Staverton,GB,ENG,<NA>,EGBJ,51.9000,-2.1667,29.0,Europe/London,1977-10-01,2022-04-13,2020-01-17,2022-04-08,NaT,NaT,37640.039822,0,-336,-1.000000,-1.0


Puling  daily  data for location  03647  and time period from  2021-05-01 00:00:00  to  2022-04-01 00:00:00
Pulling daily data


,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2021-05-01,5.5,1.3,9.6,3.1,NaN,11.0,10.4,24.1,1016.7,NaN
2021-05-02,7.2,2.0,12.1,0.0,NaN,291.0,10.4,27.8,1020.0,NaN
2021-05-03,7.0,4.5,9.2,12.3,NaN,216.0,32.8,77.8,1010.5,NaN
2021-05-04,7.6,4.2,9.9,9.8,NaN,277.0,35.7,72.2,999.4,NaN
2021-05-05,5.5,2.3,8.8,1.9,NaN,291.0,15.9,33.3,1008.4,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-03-28,9.1,4.9,13.9,0.0,NaN,68.0,8.7,22.2,1022.9,NaN
2022-03-29,7.4,5.1,9.4,0.0,NaN,49.0,15.7,29.6,1012.5,NaN
2022-03-30,5.0,2.0,7.5,0.9,NaN,26.0,16.9,48.2,1007.1,NaN


In [84]:
stations= Stations()
stations = stations.nearby(lat=51.7836, lon=-1.4854,radius=50000)
station = stations.fetch(20)

print(station)

                                            name country region    wmo  icao  \
id                                                                             
03649                               Brize Norton      GB    ENG  03649  EGVN   
EGTK0  Oxford (Kidlington) / Shipton On Cherwell      GB    ENG   <NA>  EGTK   
03647                          Little Rissington      GB    ENG  03647  <NA>   
03644              Fairford Royal Air Force Base      GB    ENG  03644  EGVA   
03658                                     Benson      GB    ENG  03658  EGUB   
EGBP0                         Kemble / Rodmarton      GB    ENG   <NA>  EGBP   
03740                                    Lyneham      GB    ENG  03740  EGDL   
EGVI0                     Greenham Common / Shaw      GB    ENG   <NA>  EGVI   
EGBJ0                Gloucestershire / Staverton      GB    ENG   <NA>  EGBJ   

       latitude  longitude  elevation       timezone hourly_start hourly_end  \
id                                     